In [1]:
#%%
import sys
# sys.path.append("/home/baokhanh/")
import pandas as pd
import numpy as np
import os
import calendar
import psycopg2
from pathlib import Path
from loguru import logger
from dotenv import load_dotenv
from datetime import datetime, timedelta
from unidecode import unidecode
from avay_bq import AvayBQServiceAcc

from slack import WebClient
from slack.errors import SlackApiError

import calendar
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from dateutil.parser import parse
from math import floor, ceil

from da_utils.client.gmail_client import GmailClient
from da_utils.repository.gmail.gmail_repository import GmailRepository
from da_utils.repository.gmail.custom_types import GmailMessage

from da_utils.client.google_sheets_client import GoogleSheetsClient
from da_utils.repository.google_sheets.google_sheets_repository import GoogleSheetsService

from da_utils.client.drive_client import GoogleDriveClient
from da_utils.repository.drive.drive_repository import GoogleDriveService

from da_utils.client.slack_client import SlackClient
from da_utils.repository.slack.slack_repository import SlackRepository

from da_utils.client.bigquery_client import BigqueryClient
from da_utils.repository.bigquery.bigquery_repository import BigqueryService

load_dotenv()
pd.options.display.max_columns = None
pd.options.display.max_colwidth = 2000

In [2]:
gmail_client = GmailClient(Path(os.environ.get("GOOGLE_CREDENTIALS_PATH")), Path(os.environ.get("GMAIL_TOKEN_PATH")))
gmail_repo = GmailRepository(gmail_client)

drive_client = GoogleDriveClient( )
drive_repo = GoogleDriveService(drive_client)

ggsheet_client = GoogleSheetsClient( )
ggsheet_repo = GoogleSheetsService(ggsheet_client)

slack_client = SlackClient(token=os.environ.get("SLACK_BOT_AUTHEN"))
slack_repo = SlackRepository(client=slack_client)

conn = psycopg2.connect(
    host = "v1proxy-new-postgres-replica.cvhqeqp3dmlv.ap-southeast-1.rds.amazonaws.com",
    dbname = "score_api_production",
    user = "analytic",
    password = "COng@tr0ngcuctacuctac",
    port = "5432"
)

bot_auth_token = os.environ.get("SLACK_BOT_AUTHEN")
user_auth_token = os.environ.get("SLACK_USER_AUTHEN")
clientSlack = WebClient(token = bot_auth_token)

channel_id = "C04MYU3L7LN"
user_id = "U0480LX5468"
linh_id = "U047QMYB9TQ"


In [3]:
def dayrange_recon(day_from_curr: int = 25) -> tuple:
    curr_time = datetime.now()
    recon_time =(curr_time - timedelta(day_from_curr))
    last_month_time = (recon_time - timedelta(30))
    last_2month_time = (recon_time - timedelta(60))
    _, end_date = calendar.monthrange(int(recon_time.strftime("%Y")), int(recon_time.strftime("%m")))

    recon_month = recon_time.strftime("%m-%Y")
    last_month = last_month_time.strftime("%Y-%m")
    last_2month = last_2month_time.strftime("%Y-%m")
    month = recon_time.strftime("%Y-%m")
    start_date = recon_time.strftime("%Y-%m") + "-" + "01"
    end_date = recon_time.strftime("%Y-%m") + "-" + str(end_date)
    return recon_month, month, recon_time, start_date, end_date, last_month, last_2month, last_month_time, last_2month_time

recon_month, month, recon_time, start_date, end_date, last_month, last_2month, last_month_time, last_2month_time = dayrange_recon()


In [4]:
def create_or_get_folder(recon_month, parent_folder_id):
    folder_check = drive_repo.list_folder_names(parent_folder_id =  parent_folder_id)
    if recon_month not in folder_check:
        folder_id = drive_repo.create_folder(folder_name= recon_month, parent_folder_id =  parent_folder_id)
    else:
        folder_id = drive_repo.get_folders(folder_name= recon_month, parent_folder_id =  parent_folder_id)
        folder_id = folder_id[0]
    return folder_id

def create_or_get_workbook(parent_folder_id, wb_name):
    wb_check = drive_repo.list_file_names(parent_folder_id= parent_folder_id)
    if wb_name not in wb_check:
        wb = ggsheet_repo.create_spreadsheet(spreadsheet_name = wb_name,parent_folder_id = parent_folder_id)
    else:
        spreadsheet_key = drive_repo.get_files(file_name = wb_name,parent_folder_id = parent_folder_id)
        spreadsheet_key = spreadsheet_key[0]
        wb = ggsheet_repo.open_spreadsheet(spreadsheet_key = spreadsheet_key)
    return wb

def create_or_update_sheet(wb, new_sheet, input_df, starting_cell: str = 'A1'):
    sheet_names = [worksheet.title for worksheet in wb.worksheets()]
    if new_sheet not in sheet_names:
        wb.add_worksheet(title=new_sheet, rows=100, cols=20)
        ws = wb.worksheet(new_sheet)
        ggsheet_repo.write_df_to_sheet(worksheet = ws, df_to_write = input_df, starting_cell=starting_cell)
    else:
        ws = wb.worksheet(new_sheet)
        ws.clear()
        ggsheet_repo.write_df_to_sheet(worksheet = ws, df_to_write = input_df, starting_cell=starting_cell)
    if 'Sheet1' in sheet_names: 
        wb.del_worksheet(wb.worksheet('Sheet1')) 
    else:
        pass
    return

parent_folder_id = '1qvIX-uNfUEgtj99uE0qQgnEsa4Ryx1vH'


In [5]:
folder_month = create_or_get_folder(recon_month = month, parent_folder_id = parent_folder_id)
lotte_check_report =create_or_get_workbook(parent_folder_id = folder_month, wb_name = f'lotte_check_report {month}')
lotte_summary = create_or_get_workbook(parent_folder_id = folder_month, wb_name = f'lotte_summary_{month}')
Lotte_detail_leads_by_sender = create_or_get_workbook(parent_folder_id = folder_month, wb_name = f'Lotte_detail_leads_by_sender_{month}')
lotte_statement = create_or_get_workbook(parent_folder_id = folder_month, wb_name = f'lotte_statement_{month}')
path = '/home/linhnguyen/04.Reconcile/03.Lotte/'

I - GET FILE FROM GMAIL

1.1. MOBIFONE

In [6]:
list_month = [month, last_month,last_2month]
mbf_lead_list = []
for months in list_month:
    QUERY_STRING=f'[TS-LOTTE-MOBIFONE] {months} has:attachment'

    gmail_messages = gmail_repo.search_messages_by_query(query=QUERY_STRING)
    for message in gmail_messages:
        attachment_pair_id = gmail_repo.query_attachments_from_email_message(message=message)
        print(attachment_pair_id[0].attachment_file_name)
        df = gmail_repo.read_excel_attachment_to_pandas(attachment_pair = attachment_pair_id[0],
                                                   password = 'lfvn2023', sheet_name = 'TS_mbf')
        df['month'] = months
        mbf_lead_list.append(df)
mbf_lead = pd.concat(mbf_lead_list)
mbf_lead

Q_2307_sent TS_mbf_sms.xlsx
Q_2306_sent TS_mbf_sms.xlsx
Q_2305_sent TS_mbf_sms.xlsx


,Q,Reason,ImportDate,Mobile2,Address,Sex,DOB,Age,SourceData,Score,Telco,SubmitData,CIC,DuplicateData,LoanDemand,FailReason,CallBackReason,FollowReason,month
0,N,Ngoài độ tuổi,2023-07-01 06:00:56.317000,14215160,Đà Nẵng,Nam,NaN,01/01/1967,TrustingSocial,685-689,mobifone,sms,CIC trắng,Không trùng data,NaN,08.2 - CIC trắng,NaN,NaN,2023-07
1,N,Hồ sơ chờ thu,2023-07-01 06:00:57.253000,14215183,Long An,Nam,NaN,31/07/1984,TrustingSocial,645-649,mobifone,sms,CIC thỏa mãn,Không trùng data,08 - Có nhu cầu,NaN,NaN,NaN,2023-07
2,N,Liên hệ quá 3 lần không được,2023-07-01 06:00:57.660000,14215218,Hồ Chí Minh,Nam,NaN,02/03/1994,TrustingSocial,650-654,mobifone,sms,CIC thỏa mãn,Không trùng data,NaN,12 - Liên hệ quá 3 lần không được,NaN,NaN,2023-07
3,N,Follow,2023-07-01 06:00:58.020000,14215244,An Giang,Nam,NaN,12/05/1984,TrustingSocial,665-669,mobifone,sms,CIC thỏa mãn,NaN,NaN,NaN,NaN,NaN,2023-07
4,N,Liên hệ quá 3 lần không được,2023-07-01 06:00:58.380000,14215270,Vĩnh Long,Nam,NaN,01/01/1987,TrustingSocial,645-649,mobifone,sms,CIC thỏa mãn,Không trùng data,NaN,12 - Liên hệ quá 3 lần không được,NaN,NaN,2023-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2369,N,Follow,2023-05-31 16:15:47.980000,14121939,Hồ Chí Minh,Nam,NaN,01/01/1978,TrustingSocial,665-669,mobifone,sms,CIC thỏa mãn,Không trùng data,NaN,NaN,NaN,04 - Chưa đủ hồ sơ,2023-05
2370,N,Liên hệ quá 3 lần không được,2023-05-31 16:15:49.923000,14121940,NaN,Nữ,NaN,NaN,TrustingSocial,815-819,mobifone,sms,NaN,NaN,NaN,12 - Liên hệ quá 3 lần không được,NaN,NaN,2023-05
2371,N,Nợ xấu,2023-05-31 16:35:47.250000,14121991,Hồ Chí Minh,NaN,NaN,21/06/1993,TrustingSocial,655-659,mobifone,sms,Nợ xấu,NaN,NaN,08.1 - CIC nợ xấu,NaN,NaN,2023-05
2372,N,Hồ sơ chờ thu,2023-05-31 16:36:02.597000,14121992,Hồ Chí Minh,Nam,NaN,05/11/1984,TrustingSocial,665-669,mobifone,sms,CIC thỏa mãn,Không trùng data,08 - Có nhu cầu,NaN,NaN,NaN,2023-05


In [7]:
mbf_lead = mbf_lead.rename(columns = {'Q': 'qualify', 'Mobile2': 'lead_id'})
mbf_lead.groupby(['month', 'Telco', 'SubmitData']).agg({'lead_id': 'count'})

,,,lead_id
month,Telco,SubmitData,
2023-05,mobifone,sms,2374
2023-06,mobifone,sms,2770
2023-07,mobifone,sms,1202


In [8]:
QUERY_STRING=f'[TS-LOTTE-MOBIFONE] {month} has:attachment'

gmail_messages = gmail_repo.search_messages_by_query(query=QUERY_STRING)

for message in gmail_messages:
    attachment_pair_id = gmail_repo.query_attachments_from_email_message(message=message)
    print(attachment_pair_id[0].attachment_file_name)
    mbf_data_disbursed = gmail_repo.read_excel_attachment_to_pandas(attachment_pair = attachment_pair_id[0],
                                                   password = 'lfvn2023', sheet_name = 'E_TS mbf', skiprows= 9, usecols=lambda x: 'Unnamed' not in x)
mbf_data_disbursed

Q_2307_sent TS_mbf_sms.xlsx


,Application No,LOANAMOUNT,DISBURSE_DATE,Import date,No of days,Fee non Vat,Fee VAT,Lead_ID
0,APL011805736,40000000,2023-07-03 00:00:00,2023-06-03 12:15:52.057000,29.488980821799487,2200000,2420000,14130769
1,APL011805711,50000000,2023-07-03 00:00:00,2023-06-26 13:40:53.217000,6.4299396181013435,2750000,3025000.0000000005,14202048
2,APL011805466,20000000,2023-07-03 00:00:00,2023-06-29 11:05:54.680000,3.537561574099527,1100000,1210000,14210768
3,APL011805225,30000000,2023-07-03 00:00:00,2023-06-06 12:01:13.900000,26.499144675901334,1650000,1815000.0000000002,14139555
4,APL011806410,20000000,2023-07-04 00:00:00,2023-06-27 08:50:59.057000,6.631260914298764,1100000,1210000,14203765
...,...,...,...,...,...,...,...,...
75,APL011818169,20000000,2023-07-28 00:00:00,2023-07-26 13:55:59.230000,1.4194533565023448,1100000,1210000,14223324
76,APL011819658,30000000,2023-07-31 00:00:00,2023-06-26 06:01:24.117000,34.74902642359666,1650000,1815000.0000000002,14198564
77,APL011818108,20000000,2023-07-31 00:00:00,2023-07-11 09:11:04.457000,19.617309525499877,1100000,1210000,14217547
78,APL011817217,20000000,2023-07-31 00:00:00,2023-06-19 06:02:02.480000,41.748582407402864,1100000,1210000,14179137


In [9]:
mbf_data_disbursed['No of days'] = mbf_data_disbursed['No of days'].astype(float).round(0).astype(int)
mbf_data_disbursed = mbf_data_disbursed[mbf_data_disbursed['No of days']<= 90]
mbf_data_disbursed.rename(columns = {'Lead_ID':'lead_id', 'PHONE':'phone_number'}, inplace = True)
mbf_data_disbursed.columns = mbf_data_disbursed.columns.str.lower()
mbf_data_disbursed.columns = mbf_data_disbursed.columns.str.replace(' ', '_')
mbf_data_disbursed

/tmp/ipykernel_1949912/3117477869.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mbf_data_disbursed.rename(columns = {'Lead_ID':'lead_id', 'PHONE':'phone_number'}, inplace = True)


,application_no,loanamount,disburse_date,import_date,no_of_days,fee_non_vat,fee_vat,lead_id
0,APL011805736,40000000,2023-07-03 00:00:00,2023-06-03 12:15:52.057000,29,2200000,2420000,14130769
1,APL011805711,50000000,2023-07-03 00:00:00,2023-06-26 13:40:53.217000,6,2750000,3025000.0000000005,14202048
2,APL011805466,20000000,2023-07-03 00:00:00,2023-06-29 11:05:54.680000,4,1100000,1210000,14210768
3,APL011805225,30000000,2023-07-03 00:00:00,2023-06-06 12:01:13.900000,26,1650000,1815000.0000000002,14139555
4,APL011806410,20000000,2023-07-04 00:00:00,2023-06-27 08:50:59.057000,7,1100000,1210000,14203765
...,...,...,...,...,...,...,...,...
75,APL011818169,20000000,2023-07-28 00:00:00,2023-07-26 13:55:59.230000,1,1100000,1210000,14223324
76,APL011819658,30000000,2023-07-31 00:00:00,2023-06-26 06:01:24.117000,35,1650000,1815000.0000000002,14198564
77,APL011818108,20000000,2023-07-31 00:00:00,2023-07-11 09:11:04.457000,20,1100000,1210000,14217547
78,APL011817217,20000000,2023-07-31 00:00:00,2023-06-19 06:02:02.480000,42,1100000,1210000,14179137


1.2. VINAPHONE

In [10]:

vnpt_lead_list = []
list_month = [month, last_month]
for months in list_month:
    CLIENT_EMAIL="anhnn5@lottefinance.vn"
    QUERY_STRING=f'[TS-LOTTE-AVAY] {months} (VNPT) from:{CLIENT_EMAIL} has:attachment'

    gmail_messages = gmail_repo.search_messages_by_query(query=QUERY_STRING)
    for message in gmail_messages:
        attachment_pair_id = gmail_repo.query_attachments_from_email_message(message=message)
        df = gmail_repo.read_excel_attachment_to_pandas(attachment_pair = attachment_pair_id[0],
                                                   password = 'lfvn2023', sheet_name = 'TS_noscore')
        df['month'] = months
        vnpt_lead_list.append(df)

QUERY_STRING=f'[TS-LOTTE-AVAY] {last_2month} (VNPT) from:{CLIENT_EMAIL} has:attachment'

gmail_messages = gmail_repo.search_messages_by_query(query=QUERY_STRING)
for message in gmail_messages:
    attachment_pair_id = gmail_repo.query_attachments_from_email_message(message=message)
    df = gmail_repo.read_excel_attachment_to_pandas(attachment_pair = attachment_pair_id[0],
                                                   password = 'lfvn2023', sheet_name = 'TS_ns')
    df['month'] = last_2month
    print(attachment_pair_id)
    vnpt_lead_list.append(df)
        
vnpt_lead = pd.concat(vnpt_lead_list)
vnpt_lead.head()

[AttachmentIdNamePair(messageId='1884111e38c15975', attachmentId='ANGjdJ_Ln4Vv_NnVIoHMtsC-JT-s7Jq6sj9aVeBJMhajDRkk7P4PiXKuCBOHrqjrBBoG2q727Ypqo3yE9PcHib-72g6GIdvPXAbvNBavGB8hn96mVQF93TLG-Tn3MRrsh9lgCk4yYW7qBwGysnFORL7DDbkXdFh6twp3ip8th6zl1gzhQRsB-6sTyMtEtQN7waslbU7izX8oFV7CUMOJcAHxnSJDUo2xoAqKILsgX6X4oIT7YMLJRpa4ZQulLtfJvYbBnhw8lRF3fXtOC7OIqzGLOgvHsqc3QJJ2X_OfNL2COraycbRTBPvfsHVLX04d_E1HtQIE57ZqxCV-S5Iu6Z6eE8amC_7MSWtWH8Gs2GNM2-QyTxXhPK_dMOpmAtMFPBCDfhbaidnuk4Bubi8s', attachment_file_name='Q_2304_sent TS_vina_noscore.xlsx')]


,Q,Reason,ImportDate,Mobile2,Address,Sex,DOB,Age,SourceData,Score,Telco,SubmitData,CIC,DuplicateData,LoanDemand,FailReason,CallBackReason,FollowReason,month
0,N,Nợ xấu,2023-06-01 06:02:47.717000,14122368,Kiên Giang,NaN,1997-01-01,01/01/1997,TrustingSocial,NaN,vinaphone,avay,Nợ xấu,Không trùng data,NaN,08.1 - CIC nợ xấu,NaN,NaN,2023-06
1,N,Liên hệ quá 3 lần không được,2023-06-01 06:04:17.507000,14122549,Hồ Chí Minh,Nam,1999-01-01,01/01/1999,TrustingSocial,NaN,vinaphone,avay,CIC thỏa mãn,Không trùng data,NaN,12 - Liên hệ quá 3 lần không được,NaN,NaN,2023-06
2,N,Nợ xấu,2023-06-01 13:00:16.640000,14124333,Bình Dương,Nữ,1993-01-01,NaN,TrustingSocial,NaN,vinaphone,avay,Nợ chú ý,NaN,NaN,08.1 - CIC nợ xấu,NaN,NaN,2023-06
3,N,Call back,2023-06-01 15:42:16.210000,14125010,NaN,Nam,1994-01-01,01/01/1994,TrustingSocial,NaN,vinaphone,avay,CIC thỏa mãn,NaN,NaN,NaN,NaN,NaN,2023-06
4,N,Liên hệ quá 3 lần không được,2023-06-02 06:03:16.207000,14125967,Hồ Chí Minh,Nam,1997-01-01,01/01/1997,TrustingSocial,NaN,vinaphone,avay,CIC thỏa mãn,NaN,NaN,12 - Liên hệ quá 3 lần không được,NaN,NaN,2023-06


In [11]:
vnpt_lead = vnpt_lead.rename(columns = {'Q': 'qualify', 'Mobile2': 'lead_id'})
vnpt_lead.groupby(['month', 'Telco', 'SubmitData']).agg({'lead_id': 'count'})

,,,lead_id
month,Telco,SubmitData,
2023-04,vinaphone,avay,1382
2023-05,vinaphone,avay,636
2023-06,vinaphone,avay,106


In [12]:
QUERY_STRING=f'[TS-LOTTE-AVAY] {month} (VNPT) from:{CLIENT_EMAIL} has:attachment'

gmail_messages = gmail_repo.search_messages_by_query(query=QUERY_STRING)

for message in gmail_messages:
    attachment_pair_id = gmail_repo.query_attachments_from_email_message(message=message)
    vnpt_data_disbursed = gmail_repo.read_excel_attachment_to_pandas(attachment_pair = attachment_pair_id[0],
                                                   password = 'lfvn2023', sheet_name = 'E_TS vina no score', skiprows= 9, usecols=lambda x: 'Unnamed' not in x)

vnpt_data_disbursed.head()

,Application No,CUSTOMER_NAME,PHONE,NATIONAL_ID,LOANAMOUNT,DISBURSE_DATE,Import date,No of days,Fee non Vat,Fee VAT,Lead_ID


In [13]:
vnpt_data_disbursed['No of days'] = vnpt_data_disbursed['No of days'].astype(int)
vnpt_data_disbursed = vnpt_data_disbursed[vnpt_data_disbursed['No of days']<= 60]
vnpt_data_disbursed.rename(columns = {'Lead_ID':'lead_id', 'PHONE':'phone_number'}, inplace = True)
vnpt_data_disbursed.columns = vnpt_data_disbursed.columns.str.lower()
vnpt_data_disbursed.columns = vnpt_data_disbursed.columns.str.replace(' ', '_')
vnpt_data_disbursed

,application_no,customer_name,phone_number,national_id,loanamount,disburse_date,import_date,no_of_days,fee_non_vat,fee_vat,lead_id


1.3 VIETTEL

In [11]:
QUERY_STRING=f'"[TS-LOTTE-AVAY] Leadgen Services Reconciliation for {month}" NOT "[TS-LOTTE-AVAY] Leadgen Services Reconciliation for {month} (VNPT)" has:attachment'

gmail_messages = gmail_repo.search_messages_by_query(query=QUERY_STRING)

for message in gmail_messages:
    attachment_pair_id = gmail_repo.query_attachments_from_email_message(message=message)
    print(attachment_pair_id[0].attachment_file_name)
    vt_lead = gmail_repo.read_excel_attachment_to_pandas(attachment_pair = attachment_pair_id[0],
                                                   password = 'lfvn2023', sheet_name = 'Sheet1')
vt_lead

Q_2307_sent TS_viettel_avay.xlsx


,Q,Reason,Score range,Channel,ImportDate,Mobile2,Address,Sex,DOB,Age,SourceData,Score,Telco,SubmitData,CIC,DuplicateData,LoanDemand,FailReason,CallBackReason,FollowReason,ID trùng
0,N,Nợ xấu,650+,viettel_avay,45108.2506520023,14215172,Hồ Chí Minh,Nữ,NaN,01/01/1995,TrustingSocial,700-709,viettel,avay,Nợ chú ý,Không trùng data,NaN,08.1 - CIC nợ xấu,NaN,NaN,NaN
1,N,Liên hệ quá 3 lần không được,630-649,viettel_avay,45108.2506623032,14215202,Hà Nội,Nam,NaN,01/02/1992,TrustingSocial,630-639,viettel,avay,CIC thỏa mãn,NaN,NaN,12 - Liên hệ quá 3 lần không được,NaN,NaN,NaN
2,N,Ngoài độ tuổi,650+,viettel_avay,45108.2509456019,14215302,Tây Ninh,Nam,NaN,21/02/2000,TrustingSocial,730-739,viettel,avay,CIC thỏa mãn,Không trùng data,NaN,12 - Liên hệ quá 3 lần không được,NaN,NaN,NaN
3,N,Nợ xấu,630-649,viettel_avay,45108.2509501157,14215307,Quảng Ninh,Nam,1986-01-01,21/03/1986,TrustingSocial,640-649,viettel,avay,Nợ xấu,Không trùng data,NaN,08.1 - CIC nợ xấu,NaN,NaN,NaN
4,N,Nợ xấu,630-649,viettel_avay,45108.2509640394,14215377,Hà Nội,Nam,NaN,01/01/1997,TrustingSocial,640-649,viettel,avay,Nợ chú ý,NaN,NaN,08.1 - CIC nợ xấu,NaN,NaN,NaN
5,N,Liên hệ quá 3 lần không được,650+,viettel_avay,45108.2512917477,14215383,Hồ Chí Minh,Nam,NaN,10/10/1980,TrustingSocial,720-729,viettel,avay,CIC thỏa mãn,Không trùng data,NaN,12 - Liên hệ quá 3 lần không được,03 - Đang nói tắt máy,NaN,NaN
6,N,Nợ xấu,650+,viettel_avay,45108.2512964468,14215423,Long An,Nam,NaN,07/10/1991,TrustingSocial,660-669,viettel,avay,Nợ chú ý,Không trùng data,NaN,08.1 - CIC nợ xấu,NaN,NaN,NaN
7,N,Nợ xấu,630-649,viettel_avay,45108.2513055208,14215458,Bến Tre,NaN,NaN,20/11/1986,TrustingSocial,640-649,viettel,avay,Nợ xấu,NaN,NaN,08.1 - CIC nợ xấu,NaN,NaN,NaN
8,N,Ngoài độ tuổi,650+,viettel_avay,45108.2516456366,14215508,Hà Nội,NaN,NaN,05/05/2000,TrustingSocial,660-669,viettel,avay,CIC trắng,NaN,NaN,NaN,NaN,NaN,NaN
9,Y,Hồ sơ bị từ chối,630-649,viettel_avay,45108.2516539699,14215546,Long An,NaN,1983-01-01,29/09/1983,TrustingSocial,640-649,viettel,avay,CIC thỏa mãn,NaN,08 - Có nhu cầu,NaN,NaN,NaN,NaN


In [12]:
vt_lead = vt_lead[vt_lead['Q'] == 'Y']
vt_lead = vt_lead.rename(columns = {'Q': 'qualify', 'Mobile2': 'lead_id'})
vt_lead.groupby([ 'Telco', 'SubmitData']).agg({'lead_id': 'count'})

,,lead_id
Telco,SubmitData,
viettel,avay,3


In [13]:
lotte_mbf = mbf_lead[['lead_id', 'DuplicateData', 'qualify', 'Reason']]
lotte_avay = vt_lead[['lead_id', 'DuplicateData', 'qualify', 'Reason']]
# lotte_vina = vnpt_lead[['lead_id', 'DuplicateData', 'qualify', 'Reason']]

lotte_mbf['telco'] = 'mobifone'
lotte_mbf['channel'] = 'sms'

lotte_avay['telco'] = 'viettel'
lotte_avay['channel'] = 'avay'

# lotte_vina['telco'] = 'vinaphone'
# lotte_vina['channel'] = 'avay'

lotte_vina = pd.DataFrame()
vnpt_data_disbursed = pd.DataFrame()

/tmp/ipykernel_1949912/3923656465.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lotte_mbf['telco'] = 'mobifone'
/tmp/ipykernel_1949912/3923656465.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lotte_mbf['channel'] = 'sms'
/tmp/ipykernel_1949912/3923656465.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_gu

In [14]:
lotte = pd.concat([lotte_mbf, lotte_avay, lotte_vina])
lotte_disbursed = pd.concat([vnpt_data_disbursed, mbf_data_disbursed])
lotte_disbursed['fee_non_vat'] = pd.to_numeric(lotte_disbursed['fee_non_vat'])
lotte_disbursed['fee_vat'] = pd.to_numeric(lotte_disbursed['fee_vat'])
lotte_disbursed['loanamount'] = pd.to_numeric(lotte_disbursed['loanamount'])
lotte_disbursed

,application_no,loanamount,disburse_date,import_date,no_of_days,fee_non_vat,fee_vat,lead_id
0,APL011805736,40000000,2023-07-03 00:00:00,2023-06-03 12:15:52.057000,29,2200000,2420000.0,14130769
1,APL011805711,50000000,2023-07-03 00:00:00,2023-06-26 13:40:53.217000,6,2750000,3025000.0,14202048
2,APL011805466,20000000,2023-07-03 00:00:00,2023-06-29 11:05:54.680000,4,1100000,1210000.0,14210768
3,APL011805225,30000000,2023-07-03 00:00:00,2023-06-06 12:01:13.900000,26,1650000,1815000.0,14139555
4,APL011806410,20000000,2023-07-04 00:00:00,2023-06-27 08:50:59.057000,7,1100000,1210000.0,14203765
...,...,...,...,...,...,...,...,...
75,APL011818169,20000000,2023-07-28 00:00:00,2023-07-26 13:55:59.230000,1,1100000,1210000.0,14223324
76,APL011819658,30000000,2023-07-31 00:00:00,2023-06-26 06:01:24.117000,35,1650000,1815000.0,14198564
77,APL011818108,20000000,2023-07-31 00:00:00,2023-07-11 09:11:04.457000,20,1100000,1210000.0,14217547
78,APL011817217,20000000,2023-07-31 00:00:00,2023-06-19 06:02:02.480000,42,1100000,1210000.0,14179137


In [15]:
lotte.loc[lotte['lead_id'].isin(lotte_disbursed['lead_id'])].reset_index()

,index,lead_id,DuplicateData,qualify,Reason,telco,channel
0,12,14215449,Không trùng data,Y,Giải ngân thành công,mobifone,sms
1,75,14216027,Không trùng data,Y,Giải ngân thành công,mobifone,sms
2,103,14216133,Không trùng data,Y,Giải ngân thành công,mobifone,sms
3,116,14216186,Không trùng data,Y,Giải ngân thành công,mobifone,sms
4,119,14216196,Không trùng data,Y,Giải ngân thành công,mobifone,sms
...,...,...,...,...,...,...,...
74,2696,14213888,Không trùng data,N,Hồ sơ chờ thu,mobifone,sms
75,329,14051629,NaN,N,Follow,mobifone,sms
76,345,14053425,NaN,N,Call back,mobifone,sms
77,1848,14102898,NaN,N,Liên hệ quá 3 lần không được,mobifone,sms


In [16]:
#check duplicate lead_id
dup_id = lotte[lotte['lead_id'].duplicated(keep=False)]
create_or_update_sheet(wb = lotte_check_report, new_sheet = 'dup_id', input_df = dup_id)

2. QUERY DATA TS

In [17]:
lotte_query = f"""select id as lead_id, phone_number, bound_code, score, telco_code,
                    to_char(sent_at AT TIME ZONE 'UTC', 'YYYY-MM-DD') as sent_date,
                    sent_at,
                    (case when ((other ->> 'sender')::text = 'sms' OR (other ->> 'sender')::text is null and (other ->> 'channel')::text is null) then 'SMS' 
                    when (other ->> 'sender')::text = 'cold_call' then 'ColdCall'
                    when (other ->> 'channel')::text = 'sms' then 'SMS' 
                    when (other ->> 'channel')::text = 'avay_sms' then 'AVAY_SMS' 
                    when (other ->> 'sender')::text = 'avay' and (other ->> 'channel')::text is null then 'AVAY' 
                    when (other ->> 'sender')::text = 'viet_tin' and (other ->> 'channel')::text is null then 'VIET_TIN' else 'NOT_NULL' end) as channel,
                    (case when (pre_scoring_data ->> 'is_qualified')::boolean = true then 1 else 0 end) as qualified,
                    (other ->> 'id_card_number') as nid
                    from phone_infos 
                    where true
                    and bank_code like 'lotte'
                    and was_sent::boolean = true"""
ts_sent = pd.read_sql(lotte_query, conn)
ts_sent

/tmp/ipykernel_1949912/2019333272.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ts_sent = pd.read_sql(lotte_query, conn)


,lead_id,phone_number,bound_code,score,telco_code,sent_date,sent_at,channel,qualified,nid
0,7177221,841218761944,LOTTE01,649,mobifone,2020-03-25,2020-03-25 23:08:05.931490,SMS,1,079069012402
1,5114457,84932540793,LOTTE03,627,mobifone,2019-04-05,2019-04-05 06:30:14.415778,SMS,1,None
2,7186227,84908331281,LOTTE01,649,mobifone,2020-03-27,2020-03-27 07:08:11.775764,SMS,1,025628842
3,7186198,84934807371,LOTTE01,644,mobifone,2020-03-27,2020-03-27 07:04:11.399777,SMS,1,025773717
4,5915248,84965240203,LOTTE04,508,viettel,2019-08-12,2019-08-12 00:31:05.699609,AVAY,1,079082007284
...,...,...,...,...,...,...,...,...,...,...
372301,14224255,84985588823,LOTTE00,-1,viettel,2023-08-14,2023-08-14 05:02:02.089355,AVAY,1,026068007524
372302,14224324,841253466899,LOTTE00,-1,vinaphone,2023-08-17,2023-08-17 04:52:32.306320,AVAY,1,079197011878
372303,14224214,841639206741,LOTTE00,-1,viettel,2023-08-12,2023-08-12 07:46:32.179256,AVAY,1,075086009395
372304,14224275,841687525441,LOTTE00,-1,viettel,2023-08-15,2023-08-15 03:10:44.843652,AVAY,1,212270245


In [18]:
ts_sent['bound_code'] = np.where((ts_sent['score'] == 650) & (ts_sent['telco_code'] == 'mobifone'), 'LOTTE02', ts_sent['bound_code'])
ts_sent['batch_month'] = ts_sent['sent_date'].str[:7]
ts_sent['sender'] = np.where(ts_sent['channel'] == 'AVAY', 'AVAY', np.where((ts_sent['channel'] == 'SMS') & (ts_sent['telco_code'] == 'viettel'), 'VIETTEL', np.where((ts_sent['channel'] == 'SMS') & (ts_sent['telco_code'] == 'mobifone'), 'MOBIFONE', 'other')))
ts_sent.head()

,lead_id,phone_number,bound_code,score,telco_code,sent_date,sent_at,channel,qualified,nid,batch_month,sender
0,7177221,841218761944,LOTTE01,649,mobifone,2020-03-25,2020-03-25 23:08:05.931490,SMS,1,079069012402,2020-03,MOBIFONE
1,5114457,84932540793,LOTTE03,627,mobifone,2019-04-05,2019-04-05 06:30:14.415778,SMS,1,None,2019-04,MOBIFONE
2,7186227,84908331281,LOTTE01,649,mobifone,2020-03-27,2020-03-27 07:08:11.775764,SMS,1,025628842,2020-03,MOBIFONE
3,7186198,84934807371,LOTTE01,644,mobifone,2020-03-27,2020-03-27 07:04:11.399777,SMS,1,025773717,2020-03,MOBIFONE
4,5915248,84965240203,LOTTE04,508,viettel,2019-08-12,2019-08-12 00:31:05.699609,AVAY,1,079082007284,2019-08,AVAY


3. JOIN TS & LOTTE DATA

In [19]:
ts_sent['lead_id'] = ts_sent['lead_id'].astype(str)
lotte['lead_id'] = lotte['lead_id'].astype(str)
lotte_final = pd.merge(lotte, ts_sent, how='left', on = 'lead_id', suffixes = ['_ltfn', '_ts'])
lotte_final

,lead_id,DuplicateData,qualify,Reason,telco,channel_ltfn,phone_number,bound_code,score,telco_code,sent_date,sent_at,channel_ts,qualified,nid,batch_month,sender
0,14215160,Không trùng data,N,Ngoài độ tuổi,mobifone,sms,84905838658,LOTTE03,686,mobifone,2023-06-30,2023-06-30 23:00:06.570785,SMS,1,048067001490,2023-06,MOBIFONE
1,14215183,Không trùng data,N,Hồ sơ chờ thu,mobifone,sms,841266897009,LOTTE02,649,mobifone,2023-06-30,2023-06-30 23:00:07.359252,SMS,1,079084016624,2023-06,MOBIFONE
2,14215218,Không trùng data,N,Liên hệ quá 3 lần không được,mobifone,sms,84937240694,LOTTE03,652,mobifone,2023-06-30,2023-06-30 23:00:07.760884,SMS,1,079094015011,2023-06,MOBIFONE
3,14215244,NaN,N,Follow,mobifone,sms,84904679433,LOTTE03,665,mobifone,2023-06-30,2023-06-30 23:00:08.112587,SMS,1,093084002452,2023-06,MOBIFONE
4,14215270,Không trùng data,N,Liên hệ quá 3 lần không được,mobifone,sms,841265937357,LOTTE02,648,mobifone,2023-06-30,2023-06-30 23:00:08.478102,SMS,1,086087001340,2023-06,MOBIFONE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6344,14121992,Không trùng data,N,Hồ sơ chờ thu,mobifone,sms,84938139544,LOTTE03,668,mobifone,2023-05-31,2023-05-31 09:35:19.140819,SMS,1,225247788,2023-05,MOBIFONE
6345,14122032,Không trùng data,N,Call back,mobifone,sms,841227273493,LOTTE03,740,mobifone,2023-05-31,2023-05-31 09:50:33.326928,SMS,1,079197023295,2023-05,MOBIFONE
6346,14215546,NaN,Y,Hồ sơ bị từ chối,viettel,avay,841656055046,LOTTE02,643,viettel,2023-06-30,2023-06-30 23:01:33.009356,AVAY,1,080083004534,2023-06,AVAY
6347,14215595,NaN,Y,Call back,viettel,avay,841633447325,LOTTE02,648,viettel,2023-06-30,2023-06-30 23:02:03.017706,AVAY,1,341928564,2023-06,AVAY


In [20]:
#check TS lead or not
not_ts_lead = lotte_final.loc[pd.isna(lotte_final['batch_month'])]
create_or_update_sheet(wb = lotte_check_report, new_sheet = 'not_ts_lead', input_df = not_ts_lead)

In [21]:
ts_sent.groupby(['batch_month','sender', 'telco_code', 'bound_code']).size().reset_index()

,batch_month,sender,telco_code,bound_code,0
0,2018-12,VIETTEL,viettel,LOTTE01,1719
1,2019-01,VIETTEL,viettel,LOTTE01,5377
2,2019-02,VIETTEL,viettel,LOTTE01,1212
3,2019-02,VIETTEL,viettel,LOTTE02,1250
4,2019-03,MOBIFONE,mobifone,LOTTE01,3092
...,...,...,...,...,...
261,2023-07,AVAY,viettel,LOTTE00,10
262,2023-07,MOBIFONE,mobifone,LOTTE02,426
263,2023-07,MOBIFONE,mobifone,LOTTE03,1083
264,2023-08,AVAY,viettel,LOTTE00,265


In [22]:
lotte_final['score'] = lotte_final['score'].astype(int)
lotte_final['score_group'] = np.where(lotte_final['bound_code'] == 'LOTTE00', 'No Score'
                                      , np.where(lotte_final['bound_code'] == 'LOTTE01', 'Medium'
                                                 , np.where(lotte_final['bound_code'] == 'LOTTE02', 'High score'
                                                            , np.where(lotte_final['bound_code'] == 'LOTTE03', 'Low', 'other'))))

lotte_final['score_range'] = np.where((lotte_final['sender'] == 'MOBIFONE') & (lotte_final['score'] >= 601) & (lotte_final['score'] <= 630), '[601-630]'
                                      , np.where((lotte_final['sender'] == 'MOBIFONE') & (lotte_final['score'] >= 631) & (lotte_final['score'] <= 649), '[631-649]'
                                                 , np.where((lotte_final['sender'] == 'MOBIFONE') & (lotte_final['score'] >= 650), '[650+]'
                                                            , np.where((lotte_final['sender'] == 'AVAY') & (lotte_final['score'] >= 581) & (lotte_final['score'] < 600), '[581-600]'
                                                                       , np.where((lotte_final['sender'] == 'AVAY') & (lotte_final['score'] >= 600) & (lotte_final['score'] < 630), '[600-629]'
                                                                                  , np.where((lotte_final['sender'] == 'AVAY') & (lotte_final['score'] >= 630), '[630+]'
                                                                                             , np.where((lotte_final['sender'] == 'AVAY') & (lotte_final['bound_code'] == 'LOTTE00'), 'no_score'
                                                                                                        , np.where((lotte_final['sender'] == 'VIETTEL') & (lotte_final['bound_code'] == 'LOTTE01'), '[600-629]'
                                                                                                                   , np.where((lotte_final['sender'] == 'VIETTEL') & (lotte_final['bound_code'] == 'LOTTE02'), '[630+]', 'other')))))))))

lotte_final.head()

,lead_id,DuplicateData,qualify,Reason,telco,channel_ltfn,phone_number,bound_code,score,telco_code,sent_date,sent_at,channel_ts,qualified,nid,batch_month,sender,score_group,score_range
0,14215160,Không trùng data,N,Ngoài độ tuổi,mobifone,sms,84905838658,LOTTE03,686,mobifone,2023-06-30,2023-06-30 23:00:06.570785,SMS,1,048067001490,2023-06,MOBIFONE,Low,[650+]
1,14215183,Không trùng data,N,Hồ sơ chờ thu,mobifone,sms,841266897009,LOTTE02,649,mobifone,2023-06-30,2023-06-30 23:00:07.359252,SMS,1,079084016624,2023-06,MOBIFONE,High score,[631-649]
2,14215218,Không trùng data,N,Liên hệ quá 3 lần không được,mobifone,sms,84937240694,LOTTE03,652,mobifone,2023-06-30,2023-06-30 23:00:07.760884,SMS,1,079094015011,2023-06,MOBIFONE,Low,[650+]
3,14215244,NaN,N,Follow,mobifone,sms,84904679433,LOTTE03,665,mobifone,2023-06-30,2023-06-30 23:00:08.112587,SMS,1,093084002452,2023-06,MOBIFONE,Low,[650+]
4,14215270,Không trùng data,N,Liên hệ quá 3 lần không được,mobifone,sms,841265937357,LOTTE02,648,mobifone,2023-06-30,2023-06-30 23:00:08.478102,SMS,1,086087001340,2023-06,MOBIFONE,High score,[631-649]


In [23]:
lotte_final.groupby(['batch_month', 'sender', 'telco_code', 'bound_code']).size().reset_index()

,batch_month,sender,telco_code,bound_code,0
0,2023-05,MOBIFONE,mobifone,LOTTE01,344
1,2023-05,MOBIFONE,mobifone,LOTTE02,421
2,2023-05,MOBIFONE,mobifone,LOTTE03,1642
3,2023-06,AVAY,viettel,LOTTE02,3
4,2023-06,MOBIFONE,mobifone,LOTTE02,716
5,2023-06,MOBIFONE,mobifone,LOTTE03,2039
6,2023-07,MOBIFONE,mobifone,LOTTE02,329
7,2023-07,MOBIFONE,mobifone,LOTTE03,855


In [24]:
#check duplicate nid
leads_time_query = """select   (other ->> 'id_card_number') as nid
                       , count(*) as num_sent
                       , max(sent_at) as recent_sent, min(sent_at) as earliest_sent
                       , array_to_string(array_agg(distinct sent_at),',') as dates_sen
                       , (DATE_PART('year', max(sent_at)::date) - DATE_PART('year', min(sent_at)::date)) * 12 +
                         (DATE_PART('month', max(sent_at)::date) - DATE_PART('month', min(sent_at)::date)) as month_diff
              from     phone_infos as pi
              where    bank_code = 'lotte'
                       and was_sent::boolean = true
                       group by 1
                       having count(*) >1
             """
leads_time_sent = pd.read_sql(leads_time_query, conn)


/tmp/ipykernel_1949912/3910054108.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  leads_time_sent = pd.read_sql(leads_time_query, conn)


In [25]:
dup_phone = lotte_final[lotte_final['phone_number'].duplicated(keep=False)]
create_or_update_sheet(wb = lotte_check_report, new_sheet = 'dup_phone', input_df = dup_phone)

In [26]:
dup_nid = lotte_final[lotte_final['nid'].duplicated(keep=False)]
create_or_update_sheet(wb = lotte_check_report, new_sheet = 'dup_nid', input_df = dup_nid)

In [27]:
# Check duplicated data: filter cases which lotte note as "trung data":
lotte_final[lotte_final['Reason'] == "Trùng data"].groupby('channel_ts').size().reset_index()

,channel_ts,0


In [28]:
lotte_final_1 = lotte_final.loc[lotte_final['lead_id'].isin(lotte_disbursed['lead_id']) | (lotte_final['telco'] == 'viettel')]
lotte_final_2 = pd.merge(lotte_final_1, lotte_disbursed, on='lead_id', how='left')

lotte_final_2.loc[lotte_final_2['telco'] == 'mobifone', 'qualify'] = 'Y'
lotte_final_2.loc[lotte_final_2['telco'] == 'mobifone', 'reason'] = 'Giải ngân thành công'
lotte_final_2.loc[lotte_final_2['telco'] == 'vinaphone', 'qualify'] = 'Y'
lotte_final_2.loc[lotte_final_2['telco'] == 'vinaphone', 'reason'] = 'Giải ngân thành công'

lotte_final_1.groupby(['telco_code', 'batch_month']).size().reset_index(name='count')


,telco_code,batch_month,count
0,mobifone,2023-05,4
1,mobifone,2023-06,31
2,mobifone,2023-07,44
3,viettel,2023-06,3


In [29]:

lotte_final_2['correct_qualified'] = lotte_final_2['qualify']
lotte_final_2.groupby(['channel_ts', 'sender', 'score_range']).size().reset_index(name='count')

lotte_final_2['price'] = np.where(lotte_final_2['correct_qualified'] == 'Y'
                                  , np.where((lotte_final_2['channel_ts'] == 'SMS') & (lotte_final_2['sender'] == 'MOBIFONE'), lotte_final_2['fee_non_vat']
                                             , np.where((lotte_final_2['channel_ts'] == 'SMS') & (lotte_final_2['sender'] == 'VIETTEL') & (lotte_final_2['score_range'] == '[581-600]'), 140000
                                                        , np.where((lotte_final_2['channel_ts'] == 'SMS') & (lotte_final_2['sender'] == 'VIETTEL') & (lotte_final_2['score_range'] == '[600-629]'), 184000
                                                                   , np.where((lotte_final_2['channel_ts'] == 'SMS') & (lotte_final_2['sender'] == 'VIETTEL') & (lotte_final_2['score_range'] == '[630+]'), 280000
                                                                              , np.where((lotte_final_2['channel_ts'] == 'AVAY') & (lotte_final_2['sender'] == 'AVAY') & (lotte_final_2['score_range'] == '[581-600]'), 140000
                                                                                         , np.where((lotte_final_2['channel_ts'] == 'AVAY') & (lotte_final_2['sender'] == 'AVAY') & (lotte_final_2['score_range'] == '[600-629]'), 168000
                                                                                                    , np.where((lotte_final_2['channel_ts'] == 'AVAY') & (lotte_final_2['sender'] == 'AVAY') & (lotte_final_2['score_range'] == '[630+]'), 168000
                                                                                                               , np.where((lotte_final_2['channel_ts'] == 'AVAY') & (lotte_final_2['sender'] == 'AVAY') & (lotte_final_2['score_range'] == 'no_score'), lotte_final_2['fee_non_vat'], 0)))))))), 0)
lotte_final_2['telco'] = np.where(lotte_final_2['bound_code'].isin(['LOTTE01', 'LOTTE02', 'LOTTE03']), lotte_final_2['telco_code'], 'Others')

lotte_final_2.loc[lotte_final_2['qualify'] == 'Y'].groupby(['telco_code', 'channel_ts', 'score_range']).agg({'price': 'sum', 'lead_id': 'count'}).reset_index()

,telco_code,channel_ts,score_range,price,lead_id
0,mobifone,SMS,[601-630],1100000.0,1
1,mobifone,SMS,[631-649],12650000.0,9
2,mobifone,SMS,[650+],116050000.0,69
3,viettel,AVAY,[630+],504000.0,3


In [30]:
lotte_final_2

,lead_id,DuplicateData,qualify,Reason,telco,channel_ltfn,phone_number,bound_code,score,telco_code,sent_date,sent_at,channel_ts,qualified,nid,batch_month,sender,score_group,score_range,application_no,loanamount,disburse_date,import_date,no_of_days,fee_non_vat,fee_vat,reason,correct_qualified,price
0,14215449,Không trùng data,Y,Giải ngân thành công,mobifone,sms,841219939811,LOTTE03,689,mobifone,2023-06-30,2023-06-30 23:00:14.907774,SMS,1,079082004862,2023-06,MOBIFONE,Low,[650+],APL011812991,50000000.0,2023-07-24 00:00:00,2023-07-01 06:01:04.790000,23.0,2750000.0,3025000.0,Giải ngân thành công,Y,2750000.0
1,14216027,Không trùng data,Y,Giải ngân thành công,mobifone,sms,841202789039,LOTTE03,653,mobifone,2023-07-05,2023-07-05 01:15:04.878981,SMS,1,049096000207,2023-07,MOBIFONE,Low,[650+],APL011807999,20000000.0,2023-07-07 00:00:00,2023-07-05 08:15:55.553000,2.0,1100000.0,1210000.0,Giải ngân thành công,Y,1100000.0
2,14216133,Không trùng data,Y,Giải ngân thành công,mobifone,sms,84903331104,LOTTE02,643,mobifone,2023-07-05,2023-07-05 04:15:06.969836,SMS,1,075081008945,2023-07,MOBIFONE,High score,[631-649],APL011809394,20000000.0,2023-07-12 00:00:00,2023-07-05 11:15:57.613000,7.0,1100000.0,1210000.0,Giải ngân thành công,Y,1100000.0
3,14216186,Không trùng data,Y,Giải ngân thành công,mobifone,sms,841204701679,LOTTE03,666,mobifone,2023-07-05,2023-07-05 07:25:04.682305,SMS,1,087185020894,2023-07,MOBIFONE,Low,[650+],APL011808479,20000000.0,2023-07-10 00:00:00,2023-07-05 14:25:55.397000,4.0,1100000.0,1210000.0,Giải ngân thành công,Y,1100000.0
4,14216196,Không trùng data,Y,Giải ngân thành công,mobifone,sms,84934898582,LOTTE03,702,mobifone,2023-07-05,2023-07-05 07:50:04.763304,SMS,1,068086005864,2023-07,MOBIFONE,Low,[650+],APL011807916,20000000.0,2023-07-13 00:00:00,2023-07-05 14:50:55.453000,7.0,1100000.0,1210000.0,Giải ngân thành công,Y,1100000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,14102898,NaN,Y,Liên hệ quá 3 lần không được,mobifone,sms,84938849193,LOTTE03,723,mobifone,2023-05-24,2023-05-24 02:36:01.270905,SMS,1,079190012912,2023-05,MOBIFONE,Low,[650+],APL011812016,30000000.0,2023-07-19 00:00:00,2023-05-24 09:36:42.897000,56.0,1650000.0,1815000.0,Giải ngân thành công,Y,1650000.0
78,14118614,NaN,Y,Follow,mobifone,sms,84937990388,LOTTE03,769,mobifone,2023-05-30,2023-05-30 06:55:40.919132,SMS,1,080081000521,2023-05,MOBIFONE,Low,[650+],APL011810433,20000000.0,2023-07-13 00:00:00,2023-05-30 13:56:23.993000,43.0,1100000.0,1210000.0,Giải ngân thành công,Y,1100000.0
79,14215546,NaN,Y,Hồ sơ bị từ chối,viettel,avay,841656055046,LOTTE02,643,viettel,2023-06-30,2023-06-30 23:01:33.009356,AVAY,1,080083004534,2023-06,AVAY,High score,[630+],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,168000.0
80,14215595,NaN,Y,Call back,viettel,avay,841633447325,LOTTE02,648,viettel,2023-06-30,2023-06-30 23:02:03.017706,AVAY,1,341928564,2023-06,AVAY,High score,[630+],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,168000.0


In [31]:
summary_total = lotte_final_2.groupby('score_range').agg({'lead_id': 'count', 'correct_qualified': lambda x: sum(x == 'Y')
                                                          , 'loanamount': 'sum', 'price': 'sum'}).rename(columns={'lead_id': 'Total_lead', 'correct_qualified': 'Qualified_leads', 'price': 'Fee_non_VAT'}).reset_index()
summary_total['Fee_icd_VAT'] = summary_total['Fee_non_VAT'] * 1.1
summary_total = summary_total.append(summary_total.agg(['sum']))


summary_avay_vt = lotte_final_2.loc[(lotte_final_2['channel_ts'] == 'AVAY') & (lotte_final_2['telco_code'] == 'viettel')
                                    ].groupby('score_range').agg({'lead_id': 'count', 'correct_qualified': lambda x: sum(x == 'Y')
                                                                  , 'price': 'sum'}).rename(columns={'lead_id': 'Total_lead', 'correct_qualified': 'qualified_leads', 'price': 'fee_non_vat'}).reset_index()
summary_avay_vt['fee_w_vat'] = summary_avay_vt['fee_non_vat'] * 1.1
summary_avay_vt = summary_avay_vt.append(summary_avay_vt.agg(['sum']))

summary_avay_vn = lotte_final_2.loc[(lotte_final_2['channel_ts'] == 'AVAY') & (lotte_final_2['telco_code'] == 'vinaphone')].groupby('score_range').agg({'correct_qualified': lambda x: sum(x == 'Y'), 'loanamount': 'sum', 'price': 'sum'}).rename(columns={'correct_qualified': 'disbursed_leads', 'price': 'fee_non_vat'}).reset_index()
summary_avay_vn['fee_w_vat'] = summary_avay_vn['fee_non_vat'] * 1.1
summary_avay_vn = summary_avay_vn.append(summary_avay_vn.agg(['sum']))

summary_mobi = lotte_final_2.loc[(lotte_final_2['channel_ts'] != 'AVAY') & (lotte_final_2['telco_code'] == 'mobifone')].groupby('batch_month').agg({'correct_qualified': lambda x: sum(x == 'Y'), 'loanamount': 'sum', 'price': 'sum'}).rename(columns={'correct_qualified': 'disbursed_leads', 'price': 'fee_non_vat'}).reset_index()
summary_mobi['fee_w_vat'] = summary_mobi['fee_non_vat'] * 1.1
summary_mobi = summary_mobi.append(summary_mobi.agg(['sum']))
summary_mobi

# summary_viettel = lotte_final_2[(lotte_final_2['channel_ts'] != 'AVAY') & (lotte_final_2['telco_code'] == 'viettel')].groupby('score_range').agg({'lead_id': 'count', 'correct_qualified': lambda x: sum(x == 'Y'), 'price': 'sum'}).rename(columns={'lead_id': 'Total_lead', 'correct_qualified': 'qualified_leads', 'price': 'fee_non_vat'})
# summary_viettel['fee_w_vat'] = summary_viettel['fee_non_vat'] * 1.1
# summary_viettel = summary_viettel.append(summary_viettel.agg(['sum']))
# summary_viettel

/tmp/ipykernel_1949912/2137283054.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summary_total = summary_total.append(summary_total.agg(['sum']))
/tmp/ipykernel_1949912/2137283054.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summary_avay_vt = summary_avay_vt.append(summary_avay_vt.agg(['sum']))
/tmp/ipykernel_1949912/2137283054.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summary_avay_vn = summary_avay_vn.append(summary_avay_vn.agg(['sum']))
/tmp/ipykernel_1949912/2137283054.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summary_mobi = summary_mobi.append(summary_mobi.agg(['sum']))


,batch_month,disbursed_leads,loanamount,fee_non_vat,fee_w_vat
0,2023-05,4,1.200000e+08,6600000.0,7260000.0
1,2023-06,31,1.045000e+09,57475000.0,63222500.0
2,2023-07,44,1.195000e+09,65725000.0,72297500.0
sum,2023-052023-062023-07,79,2.360000e+09,129800000.0,142780000.0


In [32]:
create_or_update_sheet(wb = lotte_summary, new_sheet = 'summary', input_df = summary_total)

create_or_update_sheet(wb = lotte_summary, new_sheet = 'summary_avay_vt', input_df = summary_avay_vt)

create_or_update_sheet(wb = lotte_summary, new_sheet = 'summary_avay_vn', input_df = summary_avay_vn)

create_or_update_sheet(wb = lotte_summary, new_sheet = 'summary_mobi', input_df = summary_mobi)

In [33]:
#add lead source
avay_bq_acc = AvayBQServiceAcc()

avay_df = avay_bq_acc.client.query(f"""
select case when source = 'prod.sendo' then 'Sendo'
        when source = 'prod.chotot' then 'ChoTot'
        when source = 'prod.kalapa' then 'Kalapa'
        when source = 'prod.vpp.viettel' then 'Viettel Pay Pro'
        when source = 'prod.vtp.viettel' then 'Viettel Pay'
        when utm_source like 'dir%' then utm_source
        when source = 'vaycucde.vn' then 'Vaycucde'
        when utm_source = 'fimar' then 'Fimar'
        when utm_source like '%ccesstra%' then 'Accesstrade'
        when utm_source = 'masoffer' then 'Masoffer'
        when utm_source = 'vaysieude' then 'Vaysieude'
        when utm_source = 'thomaytaichinh' then 'Thomaytaichinh'
        when utm_source = 'adpia' THEN 'Adpia'
        when utm_source like '%acebook%' or utm_source like '%fb%' or utm_source like 'chatfuel%' then 'Facebook'
        when source = 'android_app' then 'Android App'
        when utm_campaign = 'adtima' or utm_source like 'zal%' then 'Zalo'
        when utm_source = 'sales_doubler' then 'Sale Doubler'
        when source = 'prod.zalo' then 'Zalo CC'
        when utm_source like 'google_ad%' then 'Google ads'
        when utm_source = 'zns' then 'ZNS'
        when utm_source = 'mpcc' then 'mpcc'
        when utm_source like 'mgi%' then 'MGID'
        when utm_source = 'fingo' then 'Fingo'
        when utm_source = 'esmart' then 'eSmart'
        when utm_source = 'linh_dan_spa' then 'Linh Dan Spa'
        when utm_source = 'cafe_anh' then 'Cafe Anh'
        when utm_source = 'tanvd' then 'Tanvd'
        when utm_source = 'adfly' then 'Adfly'
        when utm_source = 'coccoc' then 'Coccoc'
        when utm_source in ('adskeeper','adskeeper.com') then 'Adskeeper'
        when utm_source = 'vinfin' then 'Vinfin'
        when utm_source = 'propellerads' then 'Propellerads'
        when utm_source = 'dinos' then 'Dinos'
        when utm_source = 'clickadu' then 'Clickadu'
        else 'Others' end as lead_source
    ,lead_id
  from `avay-a9925.dwh.loans` l
  left join `avay-a9925.dwh.registrations` r on r.id = l.reg_id
  left join `avay-a9925.dwh.otps` o on r.otp_id = o.id
  where l.lead_id is not null
                          ;""").result().to_arrow().to_pandas()
  
avay_df

,lead_source,lead_id
0,Adfly,9145138
1,Adfly,9658128
2,Adfly,9658112
3,Adfly,8498406
4,Vaycucde,8255594
...,...,...
3651589,Viettel Pay,11841982
3651590,Viettel Pay,11841983
3651591,Viettel Pay,11768004
3651592,Viettel Pay,11768001


In [34]:
avay_df['lead_id'] = avay_df['lead_id'].astype(str)
lotte_final_add_source = pd.merge(lotte_final_2, avay_df, how='left', on='lead_id')
lotte_final_add_source

,lead_id,DuplicateData,qualify,Reason,telco,channel_ltfn,phone_number,bound_code,score,telco_code,sent_date,sent_at,channel_ts,qualified,nid,batch_month,sender,score_group,score_range,application_no,loanamount,disburse_date,import_date,no_of_days,fee_non_vat,fee_vat,reason,correct_qualified,price,lead_source
0,14215449,Không trùng data,Y,Giải ngân thành công,mobifone,sms,841219939811,LOTTE03,689,mobifone,2023-06-30,2023-06-30 23:00:14.907774,SMS,1,079082004862,2023-06,MOBIFONE,Low,[650+],APL011812991,50000000.0,2023-07-24 00:00:00,2023-07-01 06:01:04.790000,23.0,2750000.0,3025000.0,Giải ngân thành công,Y,2750000.0,NaN
1,14216027,Không trùng data,Y,Giải ngân thành công,mobifone,sms,841202789039,LOTTE03,653,mobifone,2023-07-05,2023-07-05 01:15:04.878981,SMS,1,049096000207,2023-07,MOBIFONE,Low,[650+],APL011807999,20000000.0,2023-07-07 00:00:00,2023-07-05 08:15:55.553000,2.0,1100000.0,1210000.0,Giải ngân thành công,Y,1100000.0,NaN
2,14216133,Không trùng data,Y,Giải ngân thành công,mobifone,sms,84903331104,LOTTE02,643,mobifone,2023-07-05,2023-07-05 04:15:06.969836,SMS,1,075081008945,2023-07,MOBIFONE,High score,[631-649],APL011809394,20000000.0,2023-07-12 00:00:00,2023-07-05 11:15:57.613000,7.0,1100000.0,1210000.0,Giải ngân thành công,Y,1100000.0,NaN
3,14216186,Không trùng data,Y,Giải ngân thành công,mobifone,sms,841204701679,LOTTE03,666,mobifone,2023-07-05,2023-07-05 07:25:04.682305,SMS,1,087185020894,2023-07,MOBIFONE,Low,[650+],APL011808479,20000000.0,2023-07-10 00:00:00,2023-07-05 14:25:55.397000,4.0,1100000.0,1210000.0,Giải ngân thành công,Y,1100000.0,NaN
4,14216196,Không trùng data,Y,Giải ngân thành công,mobifone,sms,84934898582,LOTTE03,702,mobifone,2023-07-05,2023-07-05 07:50:04.763304,SMS,1,068086005864,2023-07,MOBIFONE,Low,[650+],APL011807916,20000000.0,2023-07-13 00:00:00,2023-07-05 14:50:55.453000,7.0,1100000.0,1210000.0,Giải ngân thành công,Y,1100000.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,14102898,NaN,Y,Liên hệ quá 3 lần không được,mobifone,sms,84938849193,LOTTE03,723,mobifone,2023-05-24,2023-05-24 02:36:01.270905,SMS,1,079190012912,2023-05,MOBIFONE,Low,[650+],APL011812016,30000000.0,2023-07-19 00:00:00,2023-05-24 09:36:42.897000,56.0,1650000.0,1815000.0,Giải ngân thành công,Y,1650000.0,NaN
78,14118614,NaN,Y,Follow,mobifone,sms,84937990388,LOTTE03,769,mobifone,2023-05-30,2023-05-30 06:55:40.919132,SMS,1,080081000521,2023-05,MOBIFONE,Low,[650+],APL011810433,20000000.0,2023-07-13 00:00:00,2023-05-30 13:56:23.993000,43.0,1100000.0,1210000.0,Giải ngân thành công,Y,1100000.0,NaN
79,14215546,NaN,Y,Hồ sơ bị từ chối,viettel,avay,841656055046,LOTTE02,643,viettel,2023-06-30,2023-06-30 23:01:33.009356,AVAY,1,080083004534,2023-06,AVAY,High score,[630+],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,168000.0,Google ads
80,14215595,NaN,Y,Call back,viettel,avay,841633447325,LOTTE02,648,viettel,2023-06-30,2023-06-30 23:02:03.017706,AVAY,1,341928564,2023-06,AVAY,High score,[630+],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,168000.0,Viettel Pay


In [35]:
to_write_disbursed = lotte_final_add_source.loc[lotte_final_2['correct_qualified'] == 'Y'].rename(
    columns={'sent_date': 'batch_date', 'price': 'commission', 'bound_code': 'product_code', 'disburse_date': 'disbursed_date', 
             'loanamount': 'loan_amount', 'channel_ts':'channel'}).assign(
        bank_code='lotte', 
        tenor=pd.NA, 
        fee_rate=pd.Series(np.where(lotte_final_2['telco_code'] == 'mobifone', 0.055, np.where(lotte_final_2['telco_code'] == 'vinaphone', 0.035, np.nan))), 
        disbursed_status='reconciled', 
        update_at=pd.Timestamp.now(), 
        other=pd.NA,
        ).loc[:, ['bank_code', 'lead_id', 'phone_number', 'batch_date', 'channel', 'product_code', 'nid', 
                               'telco_code', 'tenor', 'loan_amount', 'disbursed_date', 'fee_rate', 'commission','disbursed_status','update_at', 'lead_source']]

to_write_disbursed.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 82 entries, 0 to 81
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   bank_code         82 non-null     object        
 1   lead_id           82 non-null     object        
 2   phone_number      82 non-null     object        
 3   batch_date        82 non-null     object        
 4   channel           82 non-null     object        
 5   product_code      82 non-null     object        
 6   nid               82 non-null     object        
 7   telco_code        82 non-null     object        
 8   tenor             0 non-null      object        
 9   loan_amount       79 non-null     float64       
 10  disbursed_date    79 non-null     object        
 11  fee_rate          79 non-null     float64       
 12  commission        82 non-null     float64       
 13  disbursed_status  82 non-null     object        
 14  update_at         82 non-nul

In [36]:
file_disbursed_stored = f'{path}{month}/lotte_disbursed_{month}.csv'

to_write_disbursed.to_csv(file_disbursed_stored, index=False)

file_path: Path = Path(file_disbursed_stored)

# Send success message to channel
clientSlack.chat_postMessage(**{"text": f"<@{user_id}> update disbursals Lotte {recon_month}"} , channel=channel_id)

# Upload file to channel
filetype = file_path.suffix[1:]
filename = file_path.name
with open(file_path, "rb") as file:
	clientSlack.files_upload(channels=channel_id, file=file, title=filename, filetype=filetype)

In [37]:
print_df = lotte_final_2.rename(
    columns={'disburse_date': 'disbursed_date', 'loanamount': 'loan_amount', 'fee_non_vat':'fee_no_vat', 'fee_vat': 'fee_with_vat'}
    ).loc[:, ['lead_id', 'sent_date', 'channel_ts', 'telco_code', 'sender', 'score_range', 'loan_amount', 'disbursed_date', 'price']]

detail_mbf = print_df.loc[print_df['sender'] == 'MOBIFONE'].loc[:, ['lead_id', 'sent_date', 'score_range', 'loan_amount', 'disbursed_date', 'price']]
detail_avay_vt = print_df.loc[(print_df['sender'] == 'AVAY') & (print_df['telco_code'] == 'viettel')].loc[:, ['lead_id', 'sent_date', 'score_range', 'price']]
detail_avay_vn = print_df.loc[(print_df['sender'] == 'AVAY') & (print_df['telco_code'] == 'vinaphone')].loc[:, ['lead_id', 'sent_date', 'score_range', 'loan_amount', 'disbursed_date', 'price']]

In [38]:
create_or_update_sheet(wb = Lotte_detail_leads_by_sender, new_sheet = 'detail', input_df = print_df)
create_or_update_sheet(wb = lotte_statement, new_sheet = 'detail_mbf', input_df = detail_mbf)
create_or_update_sheet(wb = lotte_statement, new_sheet = 'detail_avay_vt', input_df = detail_avay_vt)
create_or_update_sheet(wb = lotte_statement, new_sheet = 'detail_avay_vn', input_df = detail_avay_vn)

mbf_disbursed_stored = f'{path}{month}/lotte_mbf_disbursed_{month}.csv'
avay_vt_disbursed_stored = f'{path}{month}/lotte_avay_vt_disbursed_{month}.csv'
avay_vn_disbursed_stored = f'{path}{month}/lotte_avay_vn_disbursed_{month}.csv'

detail_mbf.to_csv(mbf_disbursed_stored, index=False)
detail_avay_vt.to_csv(avay_vt_disbursed_stored, index=False)
detail_avay_vn.to_csv(avay_vn_disbursed_stored, index=False)

In [60]:
# drive_repo.upload_file_to_drive(file_path= Path(file_disbursed_stored), parent_folder_id= folder_month)
drive_repo.upload_file_to_drive(file_path= Path('/home/linhnguyen/04.Reconcile/03.Lotte/Rmd/lotte_avay_VT_recon_detail_2023.pdf'), parent_folder_id= folder_month)
drive_repo.upload_file_to_drive(file_path= Path('/home/linhnguyen/04.Reconcile/03.Lotte/Rmd/lotte_MBF_recon_detail_2023.pdf'), parent_folder_id= folder_month)

BrokenPipeError: [Errno 32] Broken pipe

In [39]:
lotte_mobi_report = lotte_final_2[lotte_final_2['telco'] == 'mobifone']
lotte_mobi_report['change_qualified_reason'] = 'unchanged'
lotte_mobi_report['old_qualified'] = None
lotte_mobi_report['qualify'] = np.where(lotte_mobi_report['qualify']  == 'Y', 1
                                        ,np.where(lotte_mobi_report['qualify']  == 'N', 0,2))
lotte_mobi_report['report_date'] = end_date
lotte_mobi_report['qualified_date'] = end_date
lotte_mobi_report['telco_source'] = lotte_mobi_report['telco']
lotte_mobi_report['received_date'] = pd.to_datetime(lotte_mobi_report['sent_date']).dt.date
lotte_mobi_report['loan_amount'] = lotte_mobi_report['loanamount']
lotte_mobi_report['disbursed_date'] = pd.to_datetime(lotte_mobi_report['disburse_date']).dt.date
lotte_mobi_report.head()

/tmp/ipykernel_1949912/3668785405.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lotte_mobi_report['change_qualified_reason'] = 'unchanged'
/tmp/ipykernel_1949912/3668785405.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lotte_mobi_report['old_qualified'] = None
/tmp/ipykernel_1949912/3668785405.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pand

,lead_id,DuplicateData,qualify,Reason,telco,channel_ltfn,phone_number,bound_code,score,telco_code,sent_date,sent_at,channel_ts,qualified,nid,batch_month,sender,score_group,score_range,application_no,loanamount,disburse_date,import_date,no_of_days,fee_non_vat,fee_vat,reason,correct_qualified,price,change_qualified_reason,old_qualified,report_date,qualified_date,telco_source,received_date,loan_amount,disbursed_date
0,14215449,Không trùng data,1,Giải ngân thành công,mobifone,sms,841219939811,LOTTE03,689,mobifone,2023-06-30,2023-06-30 23:00:14.907774,SMS,1,079082004862,2023-06,MOBIFONE,Low,[650+],APL011812991,50000000.0,2023-07-24 00:00:00,2023-07-01 06:01:04.790000,23.0,2750000.0,3025000.0,Giải ngân thành công,Y,2750000.0,unchanged,None,2023-07-31,2023-07-31,mobifone,2023-06-30,50000000.0,2023-07-24
1,14216027,Không trùng data,1,Giải ngân thành công,mobifone,sms,841202789039,LOTTE03,653,mobifone,2023-07-05,2023-07-05 01:15:04.878981,SMS,1,049096000207,2023-07,MOBIFONE,Low,[650+],APL011807999,20000000.0,2023-07-07 00:00:00,2023-07-05 08:15:55.553000,2.0,1100000.0,1210000.0,Giải ngân thành công,Y,1100000.0,unchanged,None,2023-07-31,2023-07-31,mobifone,2023-07-05,20000000.0,2023-07-07
2,14216133,Không trùng data,1,Giải ngân thành công,mobifone,sms,84903331104,LOTTE02,643,mobifone,2023-07-05,2023-07-05 04:15:06.969836,SMS,1,075081008945,2023-07,MOBIFONE,High score,[631-649],APL011809394,20000000.0,2023-07-12 00:00:00,2023-07-05 11:15:57.613000,7.0,1100000.0,1210000.0,Giải ngân thành công,Y,1100000.0,unchanged,None,2023-07-31,2023-07-31,mobifone,2023-07-05,20000000.0,2023-07-12
3,14216186,Không trùng data,1,Giải ngân thành công,mobifone,sms,841204701679,LOTTE03,666,mobifone,2023-07-05,2023-07-05 07:25:04.682305,SMS,1,087185020894,2023-07,MOBIFONE,Low,[650+],APL011808479,20000000.0,2023-07-10 00:00:00,2023-07-05 14:25:55.397000,4.0,1100000.0,1210000.0,Giải ngân thành công,Y,1100000.0,unchanged,None,2023-07-31,2023-07-31,mobifone,2023-07-05,20000000.0,2023-07-10
4,14216196,Không trùng data,1,Giải ngân thành công,mobifone,sms,84934898582,LOTTE03,702,mobifone,2023-07-05,2023-07-05 07:50:04.763304,SMS,1,068086005864,2023-07,MOBIFONE,Low,[650+],APL011807916,20000000.0,2023-07-13 00:00:00,2023-07-05 14:50:55.453000,7.0,1100000.0,1210000.0,Giải ngân thành công,Y,1100000.0,unchanged,None,2023-07-31,2023-07-31,mobifone,2023-07-05,20000000.0,2023-07-13


In [40]:
lotte_mobi_report_20230731 = lotte_mobi_report[['report_date', 'phone_number', 'received_date', 'score_range', 'telco_source', 'qualified_date', 'qualified', 'old_qualified', 'reason', 'change_qualified_reason', 'lead_id', 'loan_amount', 'disbursed_date']]
lotte_mobi_report_20230731.head()

,report_date,phone_number,received_date,score_range,telco_source,qualified_date,qualified,old_qualified,reason,change_qualified_reason,lead_id,loan_amount,disbursed_date
0,2023-07-31,841219939811,2023-06-30,[650+],mobifone,2023-07-31,1,None,Giải ngân thành công,unchanged,14215449,50000000.0,2023-07-24
1,2023-07-31,841202789039,2023-07-05,[650+],mobifone,2023-07-31,1,None,Giải ngân thành công,unchanged,14216027,20000000.0,2023-07-07
2,2023-07-31,84903331104,2023-07-05,[631-649],mobifone,2023-07-31,1,None,Giải ngân thành công,unchanged,14216133,20000000.0,2023-07-12
3,2023-07-31,841204701679,2023-07-05,[650+],mobifone,2023-07-31,1,None,Giải ngân thành công,unchanged,14216186,20000000.0,2023-07-10
4,2023-07-31,84934898582,2023-07-05,[650+],mobifone,2023-07-31,1,None,Giải ngân thành công,unchanged,14216196,20000000.0,2023-07-13


In [41]:
lotte_mobi_report_20230731.to_csv(f'{path}{month}/lotte_mobi_report_20230731.csv', index = False)